In [ ]:
with open('postivewords.txt') as pos:
  posbag = pos.read().splitlines()

with open('negativewords.txt') as neg:
  negbag = neg.read().splitlines()

with open('outfile.txt') as out:
  bag = out.read().splitlines()

import nltk
#nltk.download('book')
#nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import re
from nltk.tokenize import word_tokenize
import sklearn

corpus = []
for i in range(len(bag)):
    review = re.sub('[^a-zA-Z]',' ',bag[i])
    review = review.lower()
    review = review.split()
    ps = WordNetLemmatizer()
    review = [ps.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)


file1=[]
for i in range(len(corpus)):
  wt = word_tokenize(corpus[i])
  pos_score = 0
  neg_score = 0
  for w in wt:
    if w in posbag:
      pos_score = pos_score+1 
    elif w in negbag:
      neg_score = neg_score+1 
  if pos_score > neg_score : 
    rating = 1  #Positive
  elif pos_score < neg_score : 
    rating = -1 #Negative
  else : 
    rating = 0 #Neutral
  file1.append(rating)

# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1500)
X = cv.fit_transform(corpus).toarray()
y = file1

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X, y,test_size=0.20, random_state=0)

#len(X_test) - 562
#len(X_train) - 2248

from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train,y_train)

y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix
cm1= confusion_matrix(y_test, y_pred)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Fitting classifier to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10,
                                    criterion = 'entropy',
                                    random_state = 0)
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)

cm2= confusion_matrix(y_test, y_pred)

import seaborn as sns

labels = ['neg','neutral','pos']
ax = sns.heatmap(cm2, annot=True, fmt='d')
ax.set_xticklabels( labels)
ax.set_yticklabels( labels)

#SVM
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

cm3= confusion_matrix(y_test, y_pred)

#knn
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric= 'minkowski',p=2)
classifier.fit(X_train,y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

cm4= confusion_matrix(y_test, y_pred)

print(cm1)#91.4 accuracy
print(cm2)#94.8
print(cm3)#90.7
print(cm4)#88.2
  
#top5 positive and negative words used in reviews
